In [14]:
import pandas as pd
import statsmodels as sm 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import date
pd.set_option('display.float_format', '{:.2f}'.format)

# Importing Data
We'll use imported data from CPS, our home energy provider. We'll also import the daily temperatures from the UC Davis agriculture department

In [15]:
import xml.etree.ElementTree as ET
import pandas as pd

#CPS historic data is only available as a part of an XML file

# Load and parse the XML file

def parse_cps_xml(path):
    xml_file_path = path
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Extract interval readings
    data_rows = []
    for reading in root.iter():
        if reading.tag.lower().endswith("intervalreading"):
            row_data = {}
            for elem in reading.iter():
                row_data[elem.tag.split('}')[-1]] = elem.text
            data_rows.append(row_data)

    # Convert to DataFrame
    df = pd.DataFrame(data_rows)

    # Convert Unix timestamp to datetime
    df['start'] = pd.to_datetime(df['start'].astype(int), unit='s')

    return df

cps1=parse_cps_xml("/Users/lukeofthehill/repos/silly-things/home_improvement_energy_savings/CPS_Electric_15_Minute_03-31-2025_07-26-2025_20250726154824606_6601803.xml")
cps2=parse_cps_xml("/Users/lukeofthehill/repos/silly-things/home_improvement_energy_savings/CPS_Electric_15_Minute_04-05-2024_10-03-2024_20250726154734293_6601803.xml")
cps3=parse_cps_xml("/Users/lukeofthehill/repos/silly-things/home_improvement_energy_savings/CPS_Electric_15_Minute_04-11-2023_10-09-2023_20250726154546331_6601803.xml")
cps4=parse_cps_xml("/Users/lukeofthehill/repos/silly-things/home_improvement_energy_savings/CPS_Electric_15_Minute_10-02-2024_04-01-2025_20250726154807176_6601803.xml")
cps5=parse_cps_xml("/Users/lukeofthehill/repos/silly-things/home_improvement_energy_savings/CPS_Electric_15_Minute_10-08-2023_04-06-2024_20250726154700270_6601803.xml")
cps6=parse_cps_xml("/Users/lukeofthehill/repos/silly-things/home_improvement_energy_savings/CPS_Electric_15_Minute_10-13-2022_04-12-2023_20250726154008166_6601803.xml")

cps=pd.concat([cps1,cps2,cps3,cps4,cps5,cps6])
cps=cps[['start','value']]
cps=cps.rename(columns={'start':'date','value':'amount'})
cps['amount']=cps['amount'].astype(float)
cps['date']=pd.to_datetime(cps['date']).dt.date
cps=cps.groupby('date')['amount'].sum().reset_index()
cps['kwh'] = cps['amount'].astype(float) / 1000
cps.head()

,date,amount,kwh
0,2022-10-14,30755.00,30.75
1,2022-10-15,43172.00,43.17
2,2022-10-16,17759.00,17.76
3,2022-10-17,7545.00,7.54
4,2022-10-18,6611.00,6.61


In [16]:
# Importing Data
temp=pd.read_csv("/Users/lukeofthehill/repos/silly-things/home_improvement_energy_savings/US Weather Data.csv")
temp=temp[temp['zipcode']==78232] # Keeping only my home county
temp['date'] = pd.to_datetime(temp['date'], format='%Y%m%d').dt.date # Converting to an actual
temp.head()

,st_abb,st_code,county_name,fips,zipcode,date,stability,tmin,tmax,tavg
40331,TX,48,Bexar,48029,78232,2022-01-01,stable,18.88,27.32,23.10
40332,TX,48,Bexar,48029,78232,2022-01-02,stable,-0.53,25.95,12.71
40333,TX,48,Bexar,48029,78232,2022-01-03,stable,-3.64,8.60,2.48
40334,TX,48,Bexar,48029,78232,2022-01-04,stable,-3.66,12.67,4.50
40335,TX,48,Bexar,48029,78232,2022-01-05,stable,2.06,18.85,10.45


In [17]:
# Home maintenance tasks
hw=pd.DataFrame({'task':["Electrical Panel Replacement",
                            'Dryer Outlet Replacement',
                            "Battery Panel Installation",
                            "AC Maintenance: Replace Temperature Sensor",
                            "Reinsulation",
                            "AC Maintenance: New Fan",
                            "Fence Replacement / Mulch",
                            "Replaced Windows",
                            "AC Maintenance: Hard Start Kit with/without Potential Relay",
                            "New Water Heater",
                            "AC Condensation Line Clog Work"],
                'date':['2022-11-04',
                    '2022-11-15',
                    '2023-09-22',
                    '2023-12-04',
                    '2024-02-28',
                    '2024-03-28',
                    '2025-04-01',
                    '2025-04-25',
                    '2025-04-29',
                    '2025-05-20',
                    '2025-07-11']})
hw['date']=pd.to_datetime(hw['date'],format='%Y-%m-%d').dt.date
hw.head()

,task,date
0,Electrical Panel Replacement,2022-11-04
1,Dryer Outlet Replacement,2022-11-15
2,Battery Panel Installation,2023-09-22
3,AC Maintenance: Replace Temperature Sensor,2023-12-04
4,Reinsulation,2024-02-28


In [18]:
# Combinind the data
df=pd.merge(cps,temp,'left', on='date')
df=pd.merge(df,hw,'left',on='date')
df['task'].fillna('No Work',inplace=True)
dummies=pd.get_dummies(df['task'],prefix='task')
dummies.columns = dummies.columns.str.replace(' ', '')
dummies.columns = dummies.columns.str.replace(':', '')
df = pd.concat([df, dummies], axis=1)
df.head()

,date,amount,kwh,st_abb,st_code,county_name,fips,zipcode,stability,tmin,tmax,tavg,task,task_ACCondensationLineClogWork,task_ACMaintenanceHardStartKitwith/withoutPotentialRelay,task_ACMaintenanceNewFan,task_ACMaintenanceReplaceTemperatureSensor,task_BatteryPanelInstallation,task_DryerOutletReplacement,task_ElectricalPanelReplacement,task_FenceReplacement/Mulch,task_NewWaterHeater,task_NoWork,task_Reinsulation,task_ReplacedWindows
0,2022-10-14,30755.00,30.75,TX,48.00,Bexar,48029.00,78232.00,stable,20.02,32.48,26.25,No Work,False,False,False,False,False,False,False,False,False,True,False,False
1,2022-10-15,43172.00,43.17,TX,48.00,Bexar,48029.00,78232.00,stable,20.70,31.20,25.95,No Work,False,False,False,False,False,False,False,False,False,True,False,False
2,2022-10-16,17759.00,17.76,TX,48.00,Bexar,48029.00,78232.00,stable,22.65,32.31,27.48,No Work,False,False,False,False,False,False,False,False,False,True,False,False
3,2022-10-17,7545.00,7.54,TX,48.00,Bexar,48029.00,78232.00,stable,18.30,31.75,25.03,No Work,False,False,False,False,False,False,False,False,False,True,False,False
4,2022-10-18,6611.00,6.61,TX,48.00,Bexar,48029.00,78232.00,stable,14.17,21.78,17.97,No Work,False,False,False,False,False,False,False,False,False,True,False,False


In [19]:
df.columns

Index(['date', 'amount', 'kwh', 'st_abb', 'st_code', 'county_name', 'fips',
       'zipcode', 'stability', 'tmin', 'tmax', 'tavg', 'task',
       'task_ACCondensationLineClogWork',
       'task_ACMaintenanceHardStartKitwith/withoutPotentialRelay',
       'task_ACMaintenanceNewFan',
       'task_ACMaintenanceReplaceTemperatureSensor',
       'task_BatteryPanelInstallation', 'task_DryerOutletReplacement',
       'task_ElectricalPanelReplacement', 'task_FenceReplacement/Mulch',
       'task_NewWaterHeater', 'task_NoWork', 'task_Reinsulation',
       'task_ReplacedWindows'],
      dtype='object')

In [20]:
import plotly.graph_objects as go

fig = go.Figure()

# Line for kWh
fig.add_trace(go.Scatter(
    x=df['date'],
    y=df['kwh'],
    mode='lines+markers',
    name='Energy Usage (kWh)',
    yaxis='y1'
))

# Line for average temperature
fig.add_trace(go.Scatter(
    x=df['date'],
    y=df['tavg'],
    mode='lines+markers',
    name='Average Temp (°F)',
    yaxis='y2'
))

# Set up layout with dual y-axes
fig.update_layout(
    title='Daily Energy Usage and Average Temperature',
    xaxis=dict(title='Date'),
    yaxis=dict(
        title=dict(text='kWh', font=dict(color='blue')),
        tickfont=dict(color='blue')
    ),
    yaxis2=dict(
        title=dict(text='Temperature (°C)', font=dict(color='red')),
        tickfont=dict(color='red'),
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.01, y=0.99),
    hovermode='x unified'
)

fig.show()

## Modeling

In [21]:
df[df['task_ACMaintenanceNewFan']==True]['date'].iloc[0]

datetime.date(2024, 3, 28)

In [22]:
from datetime import date
import numpy as np
# df.fillna(0,inplace=True)
df=df[df['date']<date(2025, 7, 1)]

after_dt=None
def recode_task_sw(var):
    after_dt=df[df[var]==True]['date'].iloc[0]
    df[var]=np.where(df['date']>=after_dt,1,0)
recode_task_sw('task_ACMaintenanceHardStartKitwith/withoutPotentialRelay')
recode_task_sw('task_ACMaintenanceNewFan')
recode_task_sw('task_ACMaintenanceReplaceTemperatureSensor')
recode_task_sw('task_BatteryPanelInstallation')
recode_task_sw('task_Reinsulation')
recode_task_sw('task_ReplacedWindows')

df=df[['date','kwh','tmin', 'tmax', 'tavg',
       'task_ACMaintenanceHardStartKitwith/withoutPotentialRelay',
       'task_ACMaintenanceNewFan',
       'task_ACMaintenanceReplaceTemperatureSensor',
       'task_BatteryPanelInstallation',
       'task_Reinsulation',
       'task_ReplacedWindows']]

pre = df[df['date'] < date(2025, 1, 1)]
post = df[df['date'] >= date(2025, 1, 1)]



In [ ]:
import pandas as pd
import itertools
import warnings
import statsmodels.api as sm

warnings.filterwarnings("ignore")

# Make sure 'date' is datetime and set as index if not already
if pre.index.name != 'date':
    pre['date'] = pd.to_datetime(pre['date'])
    pre = pre.set_index('date')
pre = pre.asfreq('D')  # assumes daily frequency

from sklearn.preprocessing import StandardScaler

# Keep date index, drop only feature columns
features = pre.drop(columns=[])  # drop nothing; date is index

# Scale
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Recombine with date index
pre= pd.DataFrame(scaled_features, columns=features.columns, index=pre.index)



# Define endogenous and multiple exogenous variables
y = pre['kwh']

exog_vars = ['tavg',
       'task_Reinsulation']

exog = pre[exog_vars].copy()
exog = exog.astype(float)  # <-- this is the key step

# Fill any missing values
y = y.fillna(method='ffill')
exog = exog.fillna(method='ffill')

# Align both on shared index
aligned_index = y.index.intersection(exog.index)
y = y.loc[aligned_index]
exog = exog.loc[aligned_index]

# Define SARIMA parameter grid
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))

P=D=Q=range(0,2)
PDQ = list(itertools.product(p, d, q))

seasonal_period = 7  # weekly
best_aic = float("inf")
best_params = None

print("🔍 Searching for best SARIMAX model...\n")

for order in pdq:
    for seasonal_order in PDQ:
        try:
            mod = sm.tsa.statespace.SARIMAX(
                y,
                exog=exog,
                order=order,
                seasonal_order=seasonal_order + (seasonal_period,),
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            results = mod.fit(disp=False)
            if results.aic < best_aic:
                best_aic = results.aic
                best_params = (order, seasonal_order)
                print(f"✅ New Best AIC: {best_aic:.2f} | Order: {order} | Seasonal: {seasonal_order}")
        except Exception as e:
            print(f"❌ Failed for Order: {order}, Seasonal: {seasonal_order} — {e}")
            continue

if best_params:
    print("\n✅ Best SARIMAX configuration:")
    print(f"Order: {best_params[0]}, Seasonal: {best_params[1]} + ({seasonal_period})")
    print(f"AIC: {best_aic:.2f}")
else:
    print("⚠️ No valid SARIMAX configuration was found.")

🔍 Searching for best SARIMAX model...

✅ New Best AIC: 2149.50 | Order: (0, 0, 0) | Seasonal: (0, 0, 0)
✅ New Best AIC: 2064.41 | Order: (0, 0, 0) | Seasonal: (0, 0, 1)
✅ New Best AIC: 2000.69 | Order: (0, 0, 0) | Seasonal: (0, 0, 2)
✅ New Best AIC: 1953.63 | Order: (0, 0, 0) | Seasonal: (0, 1, 1)
✅ New Best AIC: 1935.31 | Order: (0, 0, 0) | Seasonal: (0, 1, 2)
✅ New Best AIC: 1929.03 | Order: (0, 0, 0) | Seasonal: (1, 0, 1)
✅ New Best AIC: 1916.85 | Order: (0, 0, 0) | Seasonal: (1, 0, 2)
✅ New Best AIC: 1684.80 | Order: (0, 0, 1) | Seasonal: (0, 0, 0)
✅ New Best AIC: 1635.98 | Order: (0, 0, 1) | Seasonal: (0, 0, 1)
✅ New Best AIC: 1596.59 | Order: (0, 0, 1) | Seasonal: (0, 0, 2)
✅ New Best AIC: 1575.16 | Order: (0, 0, 1) | Seasonal: (0, 1, 1)
✅ New Best AIC: 1566.47 | Order: (0, 0, 1) | Seasonal: (0, 1, 2)
✅ New Best AIC: 1544.89 | Order: (0, 0, 1) | Seasonal: (1, 0, 1)
✅ New Best AIC: 1538.54 | Order: (0, 0, 1) | Seasonal: (1, 0, 2)
✅ New Best AIC: 1538.02 | Order: (0, 0, 1) | Season

In [ ]:
mod=sm.tsa.statespace.SARIMAX(
                y,
                exog=exog,
                order=best_params[0],
                seasonal_order=best_params[1] + (seasonal_period,),
                enforce_stationarity=False,
                enforce_invertibility=False
            )
results = mod.fit(disp=False)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                     
=========================================================================================
Dep. Variable:                               kwh   No. Observations:                  810
Model:             SARIMAX(2, 0, 2)x(0, 0, 2, 7)   Log Likelihood                -620.855
Date:                           Sat, 26 Jul 2025   AIC                           1273.710
Time:                                   16:26:44   BIC                           1348.523
Sample:                               10-14-2022   HQIC                          1302.462
                                    - 12-31-2024                                         
Covariance Type:                             opg                                         
============================================================================================================================
                                                               coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------
tmin                                                        51.7089    197.514      0.262      0.793    -335.412     438.830
tmax                                                        52.6360    201.220      0.262      0.794    -341.748     447.020
tavg                                                      -101.1246    385.487     -0.262      0.793    -856.666     654.417
task_ACMaintenanceHardStartKitwith/withoutPotentialRelay  6.175e-11   5.27e-07      0.000      1.000   -1.03e-06    1.03e-06
task_ACMaintenanceNewFan                                     0.2051      0.257      0.797      0.425      -0.299       0.709
task_ACMaintenanceReplaceTemperatureSensor                  -0.0863      0.287     -0.300      0.764      -0.649       0.477
task_BatteryPanelInstallation                               -0.1407      0.190     -0.741      0.459      -0.513       0.232
task_Reinsulation                                           -0.2209      0.424     -0.522      0.602      -1.051       0.609
task_ReplacedWindows                                              0        nan        nan        nan         nan         nan
ar.L1                                                        1.4893      0.081     18.402      0.000       1.331       1.648
ar.L2                                                       -0.4973      0.078     -6.376      0.000      -0.650      -0.344
ma.L1                                                       -0.7577      0.088     -8.655      0.000      -0.929      -0.586
ma.L2                                                       -0.0987      0.057     -1.720      0.085      -0.211       0.014
ma.S.L7                                                     -0.0045      0.035     -0.129      0.897      -0.072       0.063
ma.S.L14                                                     0.0212      0.041      0.523      0.601      -0.058       0.101
sigma2                                                       0.2802      0.009     30.711      0.000       0.262       0.298
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              2207.17
Prob(Q):                              0.96   Prob(JB):                         0.00
Heteroskedasticity (H):               0.86   Skew:                             1.14
Prob(H) (two-sided):                  0.21   Kurtosis:                        10.85
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 6.2e+24. Standard errors may be unstable.
"""